HTML  с основных страниц забираем

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from msedge.selenium_tools import Edge, EdgeOptions
from selenium import webdriver
from requests_html import HTMLSession

session = HTMLSession()
driver = webdriver.Chrome("../DataTon-2022/chromedriver.exe")

def extract_items(item):
    try:
        atag = item.div.a
        description = (atag.text.strip())
        url  = 'https://sbermegamarket.ru/' + atag.get('href')
    except: 
        return None
    
    try:
        price_parent = item.find('div',{'class':'item-price'}).text
    except: return None
    try:
        stars = item.find('div',{'class':'rating-items__stars_filled'})
        reviews_num = item.find('div',{'class':'review-amount'}).text
    except AttributeError:
        stars=''
        reviews_num=''
    result = (description,url,price_parent,stars,reviews_num)
    return result

records = []
url ='https://sbermegamarket.ru/catalog/noutbuki/page-{}/'

for page in range(20,200):
    driver.get(url.format(page))
    soup = bs(driver.page_source, 'html.parser')
    results = soup.find_all('div',{'class':'item-block'})
    for item in results:
        record = extract_items(item)
        if record:
            records.append(record)
            
driver.close()

records_to_df = records.copy()
df_res=pd.DataFrame()
records_to_df_tuple = tuple(records_to_df)
df_res = pd.DataFrame(records_to_df_tuple,columns=['description','url','price','stars','reviews_num'])
df_res.to_csv('Sber_Mega_market_20-200_pages_laptops.csv')

фичи

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import json
from csv import reader
from tqdm import tqdm
import re

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

results_sbmm = {}
line_num = 0
with open('urls_sbmm.csv', 'r') as file:
    for line in tqdm(file.readlines()):
        try:
            url = line

            #инициализируем словари для временного храниения информаци
            seller = dict()
            parameters = dict()

            driver.get(url=url)

            #даем странице время прогрузиться
            time.sleep(2 + 2 * random.random())

            #объект bs4 для парсинга
            soup = BeautifulSoup(driver.page_source, 'lxml')
            
            #вытаскиваем параметры из таблицы "О товаре"
            product_parameters = (soup.body.main.find_all('div',class_='pdp-specs__list')
                                  )
            #
            parameters = dict()
            for table in product_parameters[0].find_all(class_='pdp-specs__group-info'):
                #print(table)
                for row in table.find_all(class_=re.compile('^pdp-specs__item-')):
                    if row['class'][0] == 'pdp-specs__item-name':
                        param_name = row.text.strip()
                    elif row['class'][0] == 'pdp-specs__item-value':
                        param_value = row.text.strip()
                        parameters[param_name] = param_value
                        
            product_url = url
            results_sbmm[product_url] = parameters
            
        except:
            continue
